# DS4DS Homework Exercise Sheet 8

**General Instructions:**

- Collaborations between students during problem-solving phase on a discussion basis is OK
- However: individual code programming and submissions per student are required
- Code sharing is strictly prohibited
- We will run checks for shared code, general plagiarism and AI-generated solutions
- Any fraud attempt will lead to an auto fail of the entire course
- Do not use any additional packages except for those provided in the task templates
- Please use Julia Version 1.10.x to ensure compatibility
- Please only write between the `#--- YOUR CODE STARTS HERE ---#` and `#--- YOUR CODE ENDS HERE ---#` comments
- Please do not delete, add any cells or overwrite cells other than the solution cells (**Tip:** If you use a jupyerhub IDE, you should not be able to add or delete cells and write in the non-solution cells by default)

## Exercise 1 - Automatic and Approximate Derivatives

As some of you might have noticed with the last exercise sheet, 
manual differentiation of complicated functions tends to be error-prone and requires
knowledge of the inner workings of said functions.
Luckily, there are ways to avoid hand-crafted derivatives.
If you have Julia functions performing tedious computations, you can use one of the many
*Automatic Differentiation*  (AD) packages.
A good AD package gives exact derivatives for differentiable functions, and oftentimes they 
employ clever tricks to do so fast and in a memory-efficient way.
In case your function relies on external software (e.g., simulations), you could try to
approximate the gradients.
Traditionally, *Finite Differences* are the go-to approach for gradient approximations.

In this exercise, we are going to implement a finite difference scheme for multi-variate functions
ourselves.
Moreover, we take a look at the AD packages `FineteDiff`, `ForwardDiff` and `Zygote`.

### Manual Finite Differences


We are interested in the (first-order) partial derivatives of $\mathcal L\colon ℝ^q \to ℝ.$

The [finite difference operator](https://en.wikipedia.org/wiki/Finite_difference) $Δ_h^{w_i}$ 
with grid-size $h > 0$ maps $\mathcal L$ to $Δ_h^{w_i}[\mathcal L]$, and we would like 
$Δ_h^{w_i}[\mathcal L](\mathbf w) \approx ∂_{w_i} \mathcal L(\mathbf w)$.
That is, the operator should approximate the partial derivative of $\mathcal L$ with respect to $w_i$ 
at $\mathbf w \in ℝ^q$.
If $\mathcal L$ is two-times continuously differentiable, then the central finite difference scheme 
is of order 2, meaning that the error of the first-order derivatives is $\mathcal O(h^2)$.

There are, of course, alternatives: forward and backward schemes, and schemes of higher 
accuracy or order, see [this table in the Wikipedia](https://en.wikipedia.org/wiki/Finite_difference_coefficient).
For now, we want to stick with the central finite diffence scheme:
$$
Δ_h^{w_i}[\mathcal L](\mathbf w) = \frac{\mathcal L(\mathbf w + h \mathbf e_i) - \mathcal L(\mathbf w - h \mathbf e_i)}{2 h}.
$$
The $i^{\text{th}}$ unit vector $\mathbf e_i$ is all zeros, except at position $i$, where it has entry $1$.

### Exercise 1.a)
Write a function `finite_diff_grad` that takes a function `func` ($\mathcal L$), 
an input vector `w` and 
a scalar grid-size `h`, and returns the central finite difference gradient approximation 
$Δ_h[\mathcal L](\mathbf w) = [Δ_h^{w_1}[\mathcal L](\mathbf w), \ldots, Δ_h^{w_q}[\mathcal L](\mathbf w)]^T \in ℝ^q$. \
Do so by completing the cell below:

In [1]:
"""
    finite_diff_grad(func, w::AbstractVector{W}, h::H=0.0001f0) where {W<:Real, H<:Real}

Return the finite difference gradient approximation of `func` at input `w`.
"""
function finite_diff_grad(func, w, h=0.0001)
    @assert h > 0 "Grid-size must be positive."

    # pre-allocate solution array:
    dw_func = zeros(length(w))
    
    # Note:
    # `zero(w)` gives an array with same element type and length
    # but above, `dw_func` is a `Vector{Float64}` to keep things simple
    
    # Now, fill `dw_func` to contain the finite difference approximations:    
    #--- YOUR CODE STARTS HERE ---#
    for i = 1:length(w)
        e_i = zeros(length(w))
        e_i[i] += 1
        dw_func[i] = (func(w + h * e_i) - func(w - h * e_i)) / (2 * h)
    end
    #--- YOUR CODE ENDS HERE ---#

    return dw_func
end

finite_diff_grad

We can test the function in the cell below:

In [2]:
let # introduce a local scope to not accidentally pollute the global environment
    func(w) = sum(w .^ 2)

    w_one = ones(2)

    dw_func = finite_diff_grad(func, w_one)
    @assert length(dw_func) == length(w_one)
    @assert dw_func ≈ [2, 2]
    
    # compute gradient of simple uni-variate function:
    @assert finite_diff_grad( w -> w[1]^2, [1.0,] ) ≈ [2.0,]

end

### Exercise 1.b)
Investigate the accuracy of `finite_diff_grad` for different grid-sizes `h` on the function
$$ \mathcal L\colon ℝ^q \to ℝ, \; \mathcal L(\mathbf w) = \exp(w_1) + \sum_{i=1}^q w^4_i,$$
where $\mathbf w = [w_1, …, w_q]^T$.

Complete the code in the cell below to test the values $h\in \{10^{-1}, 10^{-2}, …, 10^{-10}\}$.
For each grid-size value $h$, store the error
$$
\left\| Δ_h[\mathcal L](\mathbf w) - \nabla \mathcal L(\mathbf w) \right\|^2
$$
in `fd_grad_errors`. That is, besides the finite-difference approximation you also have to calculate the analytical derivative!

In [3]:
test_func(w) = exp(w[1]) + sum(w .^ 4)
function grad_test_func(w)
    global test_func
    ## return the **true** gradient vector of `test_func` w.r.t. `w`
    #--- YOUR CODE STARTS HERE ---#
    dw = zeros(length(w))
    dw[1] = exp(w[1]) + 4 * w[1]^3
    for i = 2:length(w)
        dw[i] = 4 * w[i]^3
    end
    return dw
    #--- YOUR CODE ENDS HERE ---#
end

# grid sizes ``h`` to test:
fd_grid_sizes = nothing
# Change `fd_grid_sizes` to a vector conforming to the exercise statement:
#--- YOUR CODE STARTS HERE ---#
fd_grid_sizes = zeros(10)
for i = 1:10
    fd_grid_sizes[i] = (10.)^(-i)
end
#--- YOUR CODE ENDS HERE ---#

# evaluation point, **don't** change
w_fd = [π, 10, -ℯ / 20]

# exact gradient vector `grad_test_func_exact` at `w_fd`
grad_test_func_exact = nothing
# Change `grad_test_func_exact` to hold the true gradient:
#--- YOUR CODE STARTS HERE ---#
grad_test_func_exact = grad_test_func(w_fd)
#--- YOUR CODE ENDS HERE ---#


# pre-allocate array to store the error values in
fd_grad_errors = zero(fd_grid_sizes)
for (i, h) in enumerate(fd_grid_sizes)
    ## compute finite difference gradient and store error in `fd_grad_errors[i]`
    #--- YOUR CODE STARTS HERE ---#
    fd_grad = finite_diff_grad(test_func, w_fd)
    fd_grad_errors[i] = sqrt(sum((fd_grad .- grad_test_func_exact).^2))
    #--- YOUR CODE ENDS HERE ---#
end

In [4]:
@assert length(fd_grid_sizes) == 10
@assert all(fd_grid_sizes[i] == 10.0^(-i) for i = eachindex(fd_grid_sizes))
@assert grad_test_func(zeros(4)) == [1, 0, 0, 0]
@assert sum(fd_grad_errors .^ 2) > 0

We can have a look at the error values by plotting them, for example with `Makie`.
Usually, we would expect the error to decrease initially with the grid-size.
But at some point, round-off errors will lead to an increase again.
We can see this in this pre-made plot:
![fd error plot](fd.png)

### Exercise 1.c)

#### Intro

The Julia ecosystem provides many tools for [Automatic Differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation).
`Zygote` is used in the popular machine learning library [`Flux.jl`](https://fluxml.ai/Flux.jl/stable/) 
and implements reverse accumulation to compute loss function gradients.
`ForwardDiff` is a forward-mode library and very robust. It works on functions acting on 
`Real` input.
Lastly, `FiniteDiff` does as the name suggests and computes finite difference derivative
approximations.

<div class="alert alert-block alert-info">
Flux and Lux are destined to move to Enzyme at some point in time.
The project <a href="https://github.com/JuliaDiff/AbstractDifferentiation.jl">AbstractDifferentiation</a>
wants to provide a unified API for many differentiation packages, but lacks caching mechanisms
and is thus not endorsed by SciML libraries (yet).
</div>

In [5]:
import ForwardDiff as FD
import FiniteDiff
import Zygote

Use all the above libraries to compute partial first-order derivatives of the scaled
Gaussian RBF 
$$
m(\mathbf z; a, b, \mathbf c) = b \cdot \exp\left( -\frac{\| \mathbf z - \mathbf c \|^2}{a^2} \right).
$$
The function $m$ (like $m$odel) maps $\mathbf z\in ℝ^n$ to a scalar value.
It is **c**entered at $\mathbf c \in ℝ^n$, and has shape parameter $a>0$, and scaling factor $b \in ℝ$.

In [6]:
function rbf(z, a, b, c)
    r_squared = sum((z .- c) .^ 2)
    return b * exp(-r_squared / a^2)
end

rbf (generic function with 1 method)

#### Exercise

Use `FD.gradient`, `Zygote.gradient` and `FiniteDiff.finite_difference_derivative` to
calculate the partial derivatives of $m$ with respect to $\mathbf z$,
$\mathbf c$ and $a$ respectively:
* `FD.gradient`: $\partial m / \partial \mathbf z \in \mathbb{R}^2,$
* `Zygote.gradient`: $\partial m / \partial \mathbf c \in \mathbb{R}^2,$
* `FiniteDiff.finite_difference_derivative`: $\partial m / \partial a \in \mathbb{R}.$

To do so, you can use anonymous functions, e.g. `_z -> rbf(_z, c, a)`. 
**Always return a `Vector`!**

In [7]:
function dz_rbf_ForwardDiff(z, a, b, c)
    ## compute partial derivative of `rbf` with respect to `z` using `ForwardDiff`,
    ## return a vector:
    #--- YOUR CODE STARTS HERE ---#
    dz = FD.gradient(_z -> rbf(_z, a, b, c), z)
    return dz
    #--- YOUR CODE ENDS HERE ---#
end

dz_rbf_ForwardDiff (generic function with 1 method)

In [8]:
function dc_rbf_Zygote(z, a, b, c)
    ## compute partial derivative of `rbf` with respect to `c` using `Zygote`,
    ## return a vector...

    ## Take care: `Zygote.gradient` returns a tuple for each variable array you provide as argument!
    #--- YOUR CODE STARTS HERE ---#
    dc = Zygote.gradient(_c -> rbf(z, a, b, _c), c)[1]
    return dc
    #--- YOUR CODE ENDS HERE ---#
end

dc_rbf_Zygote (generic function with 1 method)

In [9]:
function da_rbf_FiniteDiff(z, a, b, c)
    ## FiniteDiff does not like the independent variable to be an Integer...
    ## So we cast it it to `Float64`
    a = Float64(a)
    
    ## compute partial derivative of `rbf` with respect to `a` using `FiniteDiff`,
    ## return a vector...
    #--- YOUR CODE STARTS HERE ---#
    da = FiniteDiff.finite_difference_derivative(_a -> rbf(z, _a, b, c), a)
    da = [da]
    return da
    #--- YOUR CODE ENDS HERE ---#
end

da_rbf_FiniteDiff (generic function with 1 method)

We should now be able to evaluate the partial gradients at 
$(\mathbf z, \mathbf c, a) = (\mathbf 1, \mathbf 0, 1)$:

In [10]:
let z = ones(2), c = zeros(2), a = 1, b = 1
    ## evaluate partial gradient with respect to x
    @assert dz_rbf_ForwardDiff(z, a, b, c) isa Vector
    @assert length(dz_rbf_ForwardDiff(z, a, b, c)) == 2
end

In [11]:
let z = ones(2), c = zeros(2), a = 1, b = 1
    @assert dc_rbf_Zygote(z, a, b, c) isa Vector
    @assert length(dc_rbf_Zygote(z, a, b, c)) == 2
end

In [12]:
let z = ones(2), c = zeros(2), a = 1, b = 1
    @assert da_rbf_FiniteDiff(z, a, b, c) isa Vector
    @assert length(da_rbf_FiniteDiff(z, a, b, c)) == 1
end

### Exercise 1d)

#### Intro

Just like on the last exercise sheet, we now compose a complete RBF approximation model 
$h \colon ℝ^n \to ℝ$
as the sum of $N_\text{RBF} \in ℕ$ RBFs with different parameters
$$
h(\mathbf z) = 
h(\mathbf z; \mathbf w)
= \sum_{i=1}^{N_{\text{RBF}}} m(\mathbf z; a_i, b_i, \mathbf c_i).
$$
Here, $\mathbf w$ is the complete (flattened) parameter vector of $h$ holding 
$a_i, b_i$ and $\mathbf c_i$ for $i=1,…, N_{\text{RBF}}$.

In [13]:
"""
   h_rbf(z, w)

Given a flattened parameter vector `w` (of suitable size), return the value
of the complete RBF model.
"""
function h_rbf(z, w)
    dim_z = length(z)
    @assert dim_z > 0

    num_rbf_params = dim_z + 2 # (1 center + shape param + coefficient) per RBF “kernel”
    
    ## check length of `w` vector
    len_w = length(w)
    @assert len_w >= num_rbf_params
    @assert len_w % num_rbf_params == 0
    num_rbfs = div(len_w, num_rbf_params)

    ## unflatten and sum RBF kernels
    h_val = 0
    j = 1
    for i = 1:num_rbfs
        a = w[j]
        b = w[j+1]
        c = @view(w[j+2:j+1+dim_z])
        h_val += rbf(z, a, b, c)
        j += num_rbf_params
    end
    return h_val
end

h_rbf


<div class="alert alert-block alert-info">
Typically, models provided by some machine learning (like `Flux` or `Lux`), would not store 
their parameters in a flattened vector.
Not only is the model implementation more intuitive that way, it also allows for optimized
evaluation on large data sets.
For example, Flux models return `Params` objects, that usually store arrays of varying dimensions, 
dependent on the model structure.
The cool thing about `Zygote` is, that it keeps that structure when taking gradients.
So the gradient of some loss function with respect to parameters that are stored in a matrix 
is a matrix, which enables convenient updating.
<br/>
Both libraries offer tools for flattening to use external optimizers.
</div>

Now assume that $\mathbf Z \in ℝ^{n \times N}$ is a matrix, 
the columns of which hold feature vectors for labeled data.
The labels are in $\mathbf y\in ℝ^{N}$.
We want to use $h(•; \mathbf w)$ to approximate the data and the arrays induce a loss function 
$$
\mathcal L(\mathbf w) = \mathcal L(\mathbf w; \mathbf Z, \mathbf y) = \frac{1}{N}
\sum_{j=1}^{N}
(\mathbf y_j - h(\mathbf z_j, \mathbf w)) ^2
$$


Here is how to compute that value:

In [14]:
function rbf_mse(w, Z, y)
    mse_val = 0
    mse_divisor = 0
    for (zj, yj) = zip(eachcol(Z), y)
        mse_val += (h_rbf(zj, w) - yj)^2
        mse_divisor += 1
    end
    mse_val /= mse_divisor
    return mse_val
end

rbf_mse (generic function with 1 method)

#### Exercise

Now it's your turn.

Complete the cell below and use `Zygote.pullback` to compute the mean squared error **and the loss gradient**
with respect to $\mathbf w$ at the same time.
The syntax is 
```julia
result, back = Zygote.pullback( some_func, func_args )
```
and `back` is the **pullback function** that gives a tuple of gradient objects when called 
with seed `one(result)` (see lecture video on reverse-mode AD).

Oftentimes, `some_func` is actually anonymous, e.g., to compute partial gradients only.
In that case, it *can* be more convenient to use a `do`-block:
```julia
result, back = Zygote.pullback( func_arg_val1, func_arg_val2, … ) do func_arg_name1, func_arg_name2, …
    # function body acting on `func_arg_name1` etc.
    local_result
end
```
Here is the exercise:

In [15]:
function rbf_mse_and_grad(w, Z, y)
    # compute and return loss and loss gradient of `rbf_mse` with respect to w
    #--- YOUR CODE STARTS HERE ---#
    result, back = Zygote.pullback(_w -> rbf_mse(_w, Z, y), w)
    return result, back(one(result))[1]
    #--- YOUR CODE ENDS HERE ---#
end

rbf_mse_and_grad (generic function with 1 method)

Let's test the implementation:

In [16]:
import Random   ## for reproducible pseudo random numbers

In [17]:
let rng = Random.seed!(31415)
    n = 3
    N = 4

    ## random training data
    Z = rand(rng, n, N)
    y = rand(rng, N)

    ## build flattened parameter vector
    num_rbfs = 10
    w = rand(rng, (n + 2) * num_rbfs)

    ## compute loss on random data
    _L = rbf_mse(w, Z, y)

    ## compute loss and loss gradient
    L, dL = rbf_mse_and_grad(w, Z, y)

    @assert L isa Number
    @assert L == _L
    @assert dL isa Vector

    ## check consistency with ForwardDiff
    _dL = FD.gradient(_w -> rbf_mse(_w, Z, y), w)
    @assert _dL ≈ dL

end

## Exercise 2 - Momentum Descent

By extending the classical (stochastic) gradient descent update rule by a momentum term,
we can sometimes observe an improved convergence rate.
Polyak's Heavy Ball momentum is one of the best-known examples of this class of algorithms.
The parameter update for a loss 
$\mathcal L\colon ℝ^q \to ℝ, \mathbf w \mapsto \mathcal L(\mathbf w)$ 
in iteration $k\in ℕ_0$ is 
$$
\mathbf w^{(k+1)}
\leftarrow
\mathbf w^{(k)}
-
α \nabla \mathcal L(\mathbf w^{(k)})
+ 
κ
(\mathbf w^{(k)} - \mathbf w^{(k-1)}).
$$
We assume to start with $\mathbf w^{(0)}$.
The algorithm is instantiated with $\mathbf w^{(-1)} = \mathbf w^{(0)}$, i.e., without momentum in the first iteration.

To test our algorithm, we are again considering the _Rosenbrock function_:
$$
\mathcal L(\mathbf w) = (a - w_1)^2 + b(w_2 - w_1^2)^2.
$$

In [18]:
function rosenbrock_2D(w; a=1, b=100)
    return (a - w[1])^2 + b * (w[2] - w[1]^2)^2
end

rosenbrock_2D (generic function with 1 method)

### Exercise 2a)
Implement the **exact** gradient of the Rosenbrock function. You can compute it by hand or use `ForwardDiff` or `Zygote`.

In [19]:
function dw_rosenbrock_2D(w; a=1, b=100)
    #--- YOUR CODE STARTS HERE ---#
    dw1 = 4 * b * w[1]^3 + 2 * w[1] - 4 * b * w[2] * w[1] - 2 * a
    dw2 = 2 * b * (w[2] - w[1]^2)
    dw = [dw1, dw2]
    return dw
    #--- YOUR CODE ENDS HERE ---#
end

dw_rosenbrock_2D (generic function with 1 method)

The global optimum is known to be $\mathbf x = [a, a^2]^\top$ and we can check for consistency,
if the value is zero and the gradient vanishes:

In [20]:
let # introduce a local scope to not pollute the global scope by accident 
    L = rosenbrock_2D([1, 1])
    @assert iszero(L)
    dL = dw_rosenbrock_2D([1, 1])
    @assert all(iszero.(dL))

end

### Exercise 2b)

We now want to implement a training algorithm `momentum_descent` with fixed
meta-parameters.
Besides the meta parameters, the function is provided with functions
`loss`, `dw_loss` and initial parameters `w`.

Assume `loss` to return a scalar loss value when called as `loss(w)`, and `dw_loss`
to return a loss gradient vector.

Fill in the cell below according to the comments to finalize the algorithm implementation:

$$
\mathbf w^{(k+1)}
\leftarrow
\mathbf w^{(k)}
-
α \nabla \mathcal L(\mathbf w^{(k)})
+ 
κ
(\mathbf w^{(k)} - \mathbf w^{(k-1)}).
$$

In [21]:
function momentum_descent(
    loss,           # loss function
    dw_loss,        # gradient function
    w;              # initial model parameters
    num_iter=10,    # number of iterations              
    alpha=0.1f0,    # gradient stepsize
    kappa=0.1f0,    # momentum factor
)

    wk = copy(w)  # do not modify the initial vector!

    ## It is good practice to preallocate memory before any loop.
    ## Hence, initialize an object `w_prev` for the previous parameters and set 
    ## the values according to the formula for the Heavy Ball scheme.
    ## Additionally, allocate a vector `momentum` for the momentum term,
    ## i.e., the difference between two consecutive weight vectors.
    #--- YOUR CODE STARTS HERE ---#
    w_prev = zeros(size(w))
    momentum = zeros(size(w))
    #--- YOUR CODE ENDS HERE ---#

    ## Finally, do the iterations by completing the code in the for loop below.
    ## Right now, `k` can be thought to equal 0.
    ## At the end of each loop iteration, `wk` should be updated to be valid for `k`.
    ## E.g., if `num_iter==1`, we have 1 iteration, update `wk`, and return ``\mathbf w^{(1)}``.
    for k = 1:num_iter
        ## 1) Obtain a loss gradient for the current `wk`.
        ## 2) Modify `momentum`, `w_prev` and `wk` according to the Heavy Ball formulas.
        #--- YOUR CODE STARTS HERE ---#
        momentum = wk - w_prev
        w_prev = wk
        wk = wk - alpha .* dw_loss(wk) + kappa .* momentum
        #--- YOUR CODE ENDS HERE ---#
    end

    ## return last parameters
    return wk
end

momentum_descent (generic function with 1 method)

### Exercise 2c)

Minimize the Rosenbrock function with $a=1$ and $b=100$ using your momentum algorithm. Use the same method to compare the results to those of the standard gradient descent (think on how to choose the hyper-parameters to achieve this!)

Perform 100 iterations, starting at `w0_rb`:

In [22]:
num_iter_rb = 100

## some initial guess:
w0_rb = [-π / 2, ℯ / 4] # don't change!

## obtain loss function from `rosenbrock_2D` for `a=1` and `b=100` and assign it to `loss_rb`.
## `loss_rb` will be the first argument for `momentum_descent`
loss_rb = nothing    # redefine below
#--- YOUR CODE STARTS HERE ---#
loss_rb = _w -> rosenbrock_2D(_w)
#--- YOUR CODE ENDS HERE ---#

## likewise, obtain loss gradient function `dw_loss_rb` from `rosenbrock_2D` for `a=1` and `b=100`
## `dw_loss_rb` will be the second argument for `momentum_descent`
dw_loss_rb = nothing   # redefine below
#--- YOUR CODE STARTS HERE ---#
dw_loss_rb = _w -> dw_rosenbrock_2D(_w)
#--- YOUR CODE ENDS HERE ---#

## here you can see how we do 100 iterations of momentum descent:
alpha_momentum = 1e-3
kappa_momentum = 0.7
wopt_rb_momentum = momentum_descent(
    loss_rb, dw_loss_rb, w0_rb;
    num_iter=num_iter_rb, alpha=alpha_momentum, kappa=kappa_momentum
)

## exercise: compare against steepest descent!
alpha_sd = alpha_momentum
kappa_sd = nothing  # redefine below
#--- YOUR CODE STARTS HERE ---#
kappa_sd = 0
#--- YOUR CODE ENDS HERE ---#
wopt_rb_sd = momentum_descent(
    loss_rb, dw_loss_rb, w0_rb;
    num_iter=num_iter_rb, alpha=alpha_sd, kappa=kappa_sd
);

Let's check your results:

In [23]:
@assert kappa_sd isa Real

In [24]:
@assert wopt_rb_momentum isa Vector
@assert wopt_rb_sd isa Vector

loss_rb(w0_rb) ≈ 326.24283461518615
dw_loss_rb(w0_rb) ≈ [-1128.4687155349027; -357.56612863151565]

# After iteration, the loss has hopefully been reduced:
@assert loss_rb(wopt_rb_sd) < loss_rb(w0_rb)
@assert loss_rb(wopt_rb_momentum) < loss_rb(w0_rb)

If your implementation works as it should, the solution trajectories look something like this:
![momentum descent with rosenbrock](momentum_rb.png)

### Exercise 2d)

Finally, in the cell below, test several configurations of the descent algorithm for 
optimization of the Rosenbrock function $\mathcal L$ with $a=1$ and $b=100$.

For `w0_rb` from above, perform 50 iterations of momentum descent for the 
meta-parameters $(α, κ)$ in `alpha_kappa_configs`.
Among those tuples, determine the tuple `(alpha_best, kappa_best)` that achieves the smallest
function value after 50 iterations.


In [25]:
## these are the meta-parameters to investigate
alpha_kappa_configs = Iterators.product((1e-4, 1e-5, 1e-6, 1e-7), (0.6, 0.2, 0.1, 1e-3, 1e-4, 1e-6))

## at the end of this cell block, you should have assigned fitting values to this tuple:
(alpha_best, kappa_best) = (-1.0, -1.0)
#--- YOUR CODE STARTS HERE ---#
best_loss = 10000
for (alpha, kappa) in alpha_kappa_configs
    wopt = momentum_descent(loss_rb, dw_loss_rb, w0_rb;
    num_iter=50, alpha=alpha, kappa=kappa);
    if rosenbrock_2D(wopt) < best_loss
        alpha_best = alpha
        kappa_best = kappa
    end
end
#--- YOUR CODE ENDS HERE ---#

In [26]:
@assert alpha_best isa Real
@assert kappa_best isa Real

## Exercise 3 - Newton's Method

Newton's method is a root finding algorithm.
Consider the function $\mathbf m\colon ℝ^q \to ℝ^q$.
Then Newton's method tries to find $\mathbf w^* \in ℝ^q$ with $\mathbf m(\mathbf w^*) = \mathbf 0$.
The update rule is 
$$
\mathbf w_{k+1} = \mathbf w_k - \left(\nabla \mathbf m(\mathbf w_k)\right)^{-1} \mathbf m(\mathbf w_k),
\quad 
k\in \mathbb N_0,
$$
where $\nabla \mathbf m(\mathbf w_k)$ is the Jacobian of $\mathbf m$ at $\mathbf w_k$.

Instead of computing the inverse in the right-most term, rather substitute it by $\mathbf v$ and solve a linear equation system
$$
\nabla \mathbf m(\mathbf w_k) \mathbf v = - \mathbf m(\mathbf w_k)
$$
to obtain $\mathbf w_{k+1} = \mathbf v + \mathbf w_k$.

### Exercise 3a)

In the cell below, use the formulas from above to implement Newton's root finding algorithm:

In [27]:
function newton_opt(
    func,       # the function ``m`` from above
    jac_func,   # a function to obtain the jacobian of ``m``
    w0          # initial guess for the root
    ;
    num_iter=10,    # number of iterations
    abs_tol=0,      # absolute stopping tolerance
)
    ## initialize `wk`
    wk = copy(w0)
    
    for k=1:num_iter
        ## evaluate `func` at `wk` and store results in `mk`
        #--- YOUR CODE STARTS HERE ---#
        mk = func(wk)
        #--- YOUR CODE ENDS HERE ---#
        
        ## if norm squared of `wk` is <= `abs_tol`, then break:
        #--- YOUR CODE STARTS HERE ---#
        if sqrt(sum(wk.^2)) <= abs_tol
            break
        end
        #--- YOUR CODE ENDS HERE ---#

        ## compute jacobian at `wk` and store result in `d_mk`
        ## then set `v` to solve `d_mk * v = -mk`
        #--- YOUR CODE STARTS HERE ---#
        d_mk = jac_func(wk)
        v = d_mk \ -mk
        #--- YOUR CODE ENDS HERE ---#

        ## finally, update `wk` with `v` to store values for iteration `k+1`
        #--- YOUR CODE STARTS HERE ---#
        wk += v
        #--- YOUR CODE ENDS HERE ---#
    end

    return wk
end

newton_opt (generic function with 1 method)

Let us test your algorithm with a simple example. 
The Jacobian of $\mathbf m( w_1, w_2 ) = [w_1^2, w_2^2]$ is 
$$
\begin{bmatrix}
    2w_1 & 0 \\
    0 & 2w_2
\end{bmatrix}.
$$
Newton's algorithm should approximate $\mathbf w^* = [0, 0]^T$:

In [28]:
let
    ## test function:
    m = w -> w .^ 2
    ## jacobian:
    dw_m = w -> [
        2*w[1] 0;
        0 2*w[2]
    ]
    
    ## inital guess
    w0 = [1.0, -3.0]
    ## call newton root finder:
    wopt = newton_opt(m, dw_m, w0; num_iter=20, abs_tol=0)
    @assert sum(m(wopt) .^ 2) <= 1e-10

    ## test stopping criterion:
    _wopt = newton_opt(m, dw_m, w0; num_iter=20, abs_tol=1e-3)
    @assert sum(m(_wopt) .^ 2) <= 1e-3
    @assert m(wopt) <= m(wopt)
end

### Exercise 3b)

We want to use Newton's method to **minimize** the Rosenbrock function.
As an optimization algorithm for $\mathcal L\colon ℝ^q \to ℝ$, Newton's root finding scheme is applied to the function 
$$
\mathbf w \mapsto \nabla \mathcal L(\mathbf w)
$$
Hence, we need the Jacobian of the gradient of the Rosenbrock function for `newton_opt`, which is the _Hessian matrix_.
Complete the cell below to return this matrix:

In [29]:
function hess_rosenbrock_2D(w; a=1, b=100)
    #--- YOUR CODE STARTS HERE ---#
    hess = Zygote.hessian(_w -> rosenbrock_2D(_w; a, b), w)
    return hess
    #--- YOUR CODE ENDS HERE ---#
end

hess_rosenbrock_2D (generic function with 1 method)

In [30]:
@assert hess_rosenbrock_2D(ones(2)) ≈ [
    802 -400;
    -400 200
]

Now apply `newton_opt` to the problem of minimizing the Rosenbrock function:

In [31]:
m_rb = nothing
## below, override `m_rb` to a suitable function, such that Newton's method optimizes the
## Rosenbrock function with parameters `a = 2, b = 100`. 
## `m_rb` is given to `newton_opt` as the first argument.
#--- YOUR CODE STARTS HERE ---#
m_rb = _w -> dw_rosenbrock_2D(_w, a=2, b=100)
#--- YOUR CODE ENDS HERE ---#

dw_m_rb = nothing
## below, set `dw_m_rb` to a suitable function that is used as the second argument for `newton_opt`:
#--- YOUR CODE STARTS HERE ---#
dw_m_rb = _w -> hess_rosenbrock_2D(_w, a=2, b=100)
#--- YOUR CODE ENDS HERE ---#

## let's actually call the algorithm:
w_opt_newton = newton_opt(m_rb, dw_m_rb, w0_rb);

In [32]:
@assert w_opt_newton isa Vector
@assert length(w_opt_newton) == 2

Now, the solution trajectory should reach the optimum quickly:
![newton iterations](newton.png)